In [1]:
 %load_ext sql

In [2]:
%sql sqlite:///library.db

'Connected: @library.db'

In [305]:
%%sql


DROP TABLE IF EXISTS CD;
DROP TABLE IF EXISTS CD_Detail;
DROP TABLE IF EXISTS Books;
DROP TABLE IF EXISTS Book_Detail;
DROP TABLE IF EXISTS Scientific_Journals;
DROP TABLE IF EXISTS SJ_Detail;
DROP TABLE IF EXISTS Magazines;
DROP TABLE IF EXISTS Magazine_Detail;
DROP TABLE IF EXISTS People;
DROP TABLE IF EXISTS Items;
DROP TABLE IF EXISTS Users;
DROP TABLE IF EXISTS Personels;
DROP TABLE IF EXISTS Borrowing;
DROP TABLE IF EXISTS Request;
DROP TABLE IF EXISTS Request_by;
DROP TABLE IF EXISTS Event;
DROP TABLE IF EXISTS Event_Detail;
DROP TABLE IF EXISTS Hosted_by;
DROP TABLE IF EXISTS Joining;


/*****************
*	Items
*****************/
CREATE TABLE IF NOT EXISTS Items(
    id INT PRIMARY KEY,
    availability VARCHAR(8),
    title VARCHAR(50),
    releaseDate DATE,
    CHECK (availability = 'available' 
		OR availability = 'borrowed' 
		OR availability = 'coming')
);


CREATE TABLE IF NOT EXISTS CD(
    	id INT PRIMARY KEY,
    	ISRC VARCHAR(50),
    	FOREIGN KEY (id) REFERENCES Items(id)
);

CREATE TABLE IF NOT EXISTS CD_Detail(
    	ISRC VARCHAR(50) PRIMARY KEY,
    	artist VARCHAR(50),
    	publisher VARCHAR(50),
	genre VARCHAR(30)
);

CREATE TABLE IF NOT EXISTS Magazines(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id)
);

CREATE TABLE IF NOT EXISTS Magazine_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
	genre VARCHAR(30),
	publisher VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Scientific_Journals(
	id INT PRIMARY KEY,
	ISSN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id)
);

CREATE TABLE IF NOT EXISTS SJ_Detail(
	ISSN VARCHAR(50) PRIMARY KEY,
	field VARCHAR(30),
	researcher VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS Books(
	id INT PRIMARY KEY,
	ISBN VARCHAR(50),
	FOREIGN KEY (id) REFERENCES Items(id)
);

CREATE TABLE Book_Detail(
	ISBN VARCHAR(50) PRIMARY KEY,
	author VARCHAR(50),
	genre VARCHAR(30),
	type VARCHAR(7),
	CHECK (type = 'printed' 
		OR type = 'online')
);
/*****************
*	People
*****************/
CREATE TABLE IF NOT EXISTS People (
	pid INT PRIMARY KEY,
	firstName VARCHAR(50),
	lastName VARCHAR(50),
	address VARCHAR(256),
	email VARCHAR(50),
	phone VARCHAR(10)
);

CREATE TABLE IF NOT EXISTS Users (
	pid INT PRIMARY KEY,
	totalFine INT,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Personels (
	pid INT PRIMARY KEY,
	salary REAL,
	position VARCHAR(30),
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

/*****************
*	Event
*****************/
CREATE TABLE IF NOT EXISTS Event (
	eid INT PRIMARY KEY,
	event INT,
    room VARCHAR(30),
	eventDate DATE,
	eventTime TIME
);

CREATE TABLE IF NOT EXISTS Event_Detail (
    event INT PRIMARY KEY,
    name VARCHAR (50) ,
	type VARCHAR (30),
	description VARCHAR(300),
	audiences VARCHAR(30)
);


/****************
*	Request
*****************/
CREATE TABLE IF NOT EXISTS Request (
	rid INT PRIMARY KEY,
	description VARCHAR(300)
);

/*****************
*	Relationships
*****************/
CREATE TABLE IF NOT EXISTS Borrowing (
	id INT,
	uid INT,
	dateBorrowed DATE,
	dueDate DATE,
	fine INT,
	dateReturned DATE,
	PRIMARY KEY (id, uid),
	FOREIGN KEY (id) REFERENCES Items(id)
	ON DELETE CASCADE,
	FOREIGN KEY (id) REFERENCES Users(pid)
	ON DELETE CASCADE,
	CHECK (dateBorrowed < dueDate 
		OR dateBorrowed < dateReturned)
);

CREATE TABLE IF NOT EXISTS Request_by (
	rid INT,
	pid INT,
	PRIMARY KEY(rid,pid),
	FOREIGN KEY (rid) REFERENCES Request(rid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Hosted_by (
	eid INT,
	pid INT,
	PRIMARY KEY(eid,pid),
	FOREIGN KEY (eid) REFERENCES Event(eid)
	ON DELETE CASCADE,
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE
);

CREATE TABLE IF NOT EXISTS Joining (
	eid INT,
	pid INT,
	PRIMARY KEY(pid, eid),
	FOREIGN KEY (pid) REFERENCES People(pid)
	ON DELETE CASCADE,
	FOREIGN KEY (eid) REFERENCES Event(eid)
	ON DELETE CASCADE
);



 * sqlite:///library.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [296]:
%%sql

/*****************
*	Triggers
*****************/
--When inserted to the Borrowing table, sets dateBorrowed and dueDate when not set
CREATE TRIGGER IF NOT EXISTS borrowDate AFTER INSERT ON Borrowing
FOR EACH ROW
WHEN (NEW.dateBorrowed IS NULL)
BEGIN 
    UPDATE Borrowing SET dateBorrowed = DATE('now') WHERE id = NEW.id;
    UPDATE Borrowing SET dueDATE = DATE('now','+14 day') WHERE dueDate IS NULL AND id = NEW.id;
END;

CREATE TRIGGER IF NOT EXISTS fine AFTER UPDATE ON Borrowing
FOR EACH ROW
WHEN (NEW.dateReturned IS NOT NULL AND NEW.dueDate IS NOT NULL)
BEGIN
    UPDATE Borrowing SET fine = JulianDay(NEW.dateReturned) - JulianDay(NEW.dueDate) WHERE id = NEW.id;
END;

/*
CREATE TRIGGER IF NOT EXISTS fine AFTER UPDATE ON Borrowing
FOR EACH ROW
WHEN (NEW.dateReturned IS NOT NULL AND NEW.dueDate IS NOT NULL)
BEGIN
    SET NEW.fine = 
    case 
    when julianday(NEW.dateReturned) - julianday(NEW.dueDate) < 0 THEN 0
    else julianday(NEW.dateReturned) - julianday(NEW.dueDate)
    END WHERE id = NEW.id;
END;
*/

 * sqlite:///library.db
Done.
Done.
0 rows affected.


[]

In [306]:
%%sql

/*****************
*	Insert
*****************/

INSERT INTO People (pid, firstName, lastName, address, email, phone)
    VALUES (123,'Nikola', 'Maeda', '4306 Triumph st', 'nikobox', '323');
INSERT INTO Users (pid, totalFine) VALUES (123, 0);
INSERT INTO Items (id) VALUES (1);
INSERT INTO Books (id) VALUES (1);
INSERT INTO Borrowing(id, uid) VALUES (1,123);
UPDATE Borrowing SET dateReturned = DATE('2019-06-15') WHERE id = 1;

 * sqlite:///library.db
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

In [307]:
%%sql




 * sqlite:///library.db


'Connected: @library.db'

In [308]:
%%sql
select * from Borrowing;

 * sqlite:///library.db
Done.


id,uid,dateBorrowed,dueDate,fine,dateReturned
1,123,None,None,None,2019-06-15
